# Professional Report Generation

This notebook demonstrates how to create **publication-ready reports** from PanelBox results using pandas export functions.

## What You'll Learn

- ✅ Use the `.summary()` method for text summaries
- ✅ Extract results as pandas DataFrames
- ✅ Generate HTML reports (interactive)
- ✅ Export to Markdown (GitHub, documentation)
- ✅ Create LaTeX tables (academic papers)
- ✅ Build comparison tables (multiple models)
- ✅ Customize formatting and styles
- ✅ Automate reporting workflows

## Table of Contents

1. [Introduction](#introduction)
2. [Basic Reporting](#basic)
3. [Exporting to Different Formats](#html)
4. [Markdown Export](#markdown)
5. [LaTeX Tables](#latex)
6. [Comparison Tables](#comparison)
7. [Custom Formatting](#custom)
8. [Automated Workflows](#automation)

---

## 1. Introduction to Report Generation {#introduction}

### Why Professional Reports Matter

Good reports:
- ✅ Communicate results clearly
- ✅ Save time (automated formatting)
- ✅ Look professional (publication-ready)
- ✅ Are reproducible (code → output)

### Report Formats

| Format | Use Case | Advantages |
|--------|----------|------------|
| **HTML** | Interactive exploration, web | Interactive, embedded plots, styling |
| **Markdown** | GitHub, docs, READMEs | Simple, version control friendly |
| **LaTeX** | Academic papers, journals | Publication-quality, precise formatting |

### What This Notebook Shows

- The `.summary()` method for text output
- Using pandas to extract and export results
- Creating HTML, Markdown, and LaTeX tables
- Building comparison tables for multiple models
- Automating report generation

Let's start!

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import panelbox as pb

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.random.seed(42)

plt.style.use('seaborn-v0_8-darkgrid')

print(f"PanelBox version: {pb.__version__}")
print("Report generation ready!")

PanelBox version: 0.8.0
Report generation ready!


In [2]:
# Load data and estimate multiple models
data = pb.load_grunfeld()

print("Estimating models for reporting...")

# Model 1: Pooled OLS
pooled = pb.PooledOLS(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
pooled_results = pooled.fit()

# Model 2: Fixed Effects
fe = pb.FixedEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
fe_results = fe.fit()

# Model 3: Random Effects
re = pb.RandomEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
re_results = re.fit()

# Model 4: Fixed Effects with robust SE
fe_robust = pb.FixedEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
fe_robust_results = fe_robust.fit(cov_type='clustered', cluster_entity=True)

print("✓ 4 models estimated")

Estimating models for reporting...
✓ 4 models estimated


---

## 2. Basic Reporting {#basic}

### 2.1 Summary Method

Every model has a `.summary()` method:

In [3]:
# Basic summary
print("FIXED EFFECTS MODEL - Basic Summary")
print("="*70)
print(fe_results.summary())

FIXED EFFECTS MODEL - Basic Summary
                       Fixed Effects Estimation Results                       
Formula: invest ~ value + capital
Model:   Fixed Effects
------------------------------------------------------------------------------
No. Observations:                 200
No. Entities:                      10
No. Time Periods:                  20
Degrees of Freedom:               188
R-squared:                     0.7899
Adj. R-squared:                0.7776
R-squared (within):            0.7899
R-squared (between):           1.0000
R-squared (overall):           0.9496
Standard Errors:            nonrobust
F-statistic (FE vs OLS):      58.2744
F-test p-value:                0.0000
Variable        Coef.        Std.Err.     t        P>|t|    [0.025     0.975]    
------------------------------------------------------------------------------
value                0.1066      0.0113   9.401  0.0000    0.0842    0.1289 ***
capital              0.3444      0.0178  19.397  0.0

### 2.2 Custom Title for Summary

You can provide a custom title to the summary method:

In [4]:
# Custom title
print("\nCUSTOM TITLE SUMMARY")
print("="*70)
print(fe_results.summary(title="Investment Equation - Fixed Effects Results"))


CUSTOM TITLE SUMMARY
                 Investment Equation - Fixed Effects Results                  
Formula: invest ~ value + capital
Model:   Fixed Effects
------------------------------------------------------------------------------
No. Observations:                 200
No. Entities:                      10
No. Time Periods:                  20
Degrees of Freedom:               188
R-squared:                     0.7899
Adj. R-squared:                0.7776
R-squared (within):            0.7899
R-squared (between):           1.0000
R-squared (overall):           0.9496
Standard Errors:            nonrobust
F-statistic (FE vs OLS):      58.2744
F-test p-value:                0.0000
Variable        Coef.        Std.Err.     t        P>|t|    [0.025     0.975]    
------------------------------------------------------------------------------
value                0.1066      0.0113   9.401  0.0000    0.0842    0.1289 ***
capital              0.3444      0.0178  19.397  0.0000    0.3094 

### 2.3 Extracting Results as DataFrames

Get results as pandas DataFrames for custom processing:

In [5]:
# Extract as DataFrame
results_df = pd.DataFrame({
    'Coefficient': fe_results.params,
    'Std Error': fe_results.std_errors,
    't-statistic': fe_results.tvalues,
    'P-value': fe_results.pvalues
})

print("\nRESULTS AS DATAFRAME")
print("="*70)
print(results_df)

print("\n💡 Now you can manipulate/export as needed!")


RESULTS AS DATAFRAME
         Coefficient  Std Error  t-statistic  P-value
value         0.1066     0.0113       9.4014      0.0
capital       0.3444     0.0178      19.3974      0.0

💡 Now you can manipulate/export as needed!


---

## 3. Exporting to Different Formats {#html}

While PanelBox provides the `.summary()` method for text output, you can easily create your own export functions using pandas and the results attributes.

### 3.1 Creating HTML Tables

You can convert results to pandas DataFrame and use pandas' HTML export:

In [6]:
# Create DataFrame from results
results_df = pd.DataFrame({
    'Coefficient': fe_results.params,
    'Std Error': fe_results.std_errors,
    't-statistic': fe_results.tvalues,
    'P-value': fe_results.pvalues
})

# Convert to HTML using pandas
html_output = results_df.to_html(float_format=lambda x: f'{x:.4f}')

print("HTML Table Generated!")
print("="*70)
print(f"Preview (first 500 chars):")
print(html_output[:500])
print("...")

# To save to file:
# with open('fe_results.html', 'w') as f:
#     f.write(html_output)
print("\n💡 Tip: Use df.to_html() to generate HTML tables")

HTML Table Generated!
Preview (first 500 chars):
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Coefficient</th>
      <th>Std Error</th>
      <th>t-statistic</th>
      <th>P-value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>value</th>
      <td>0.1066</td>
      <td>0.0113</td>
      <td>9.4014</td>
      <td>0.0000</td>
    </tr>
    <tr>
      <th>capital</th>
      <td>0.3444</td>
      <td>0.0178</td>
      <td>19.3974</td>
      <td>0.0000</td>
    </tr>
  </tbody
...

💡 Tip: Use df.to_html() to generate HTML tables


### 3.2 HTML with Custom Styling

You can add CSS to make the table look professional:

In [7]:
# Custom CSS styling
custom_css = """<style>
    table {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        border-collapse: collapse;
        width: 100%;
        max-width: 800px;
        margin: 20px auto;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }
    th {
        background-color: #2c3e50;
        color: white;
        padding: 12px;
        text-align: left;
        font-weight: bold;
    }
    td {
        padding: 10px;
        border-bottom: 1px solid #ddd;
    }
    tr:hover {
        background-color: #f5f5f5;
    }
</style>
"""

html_styled = custom_css + results_df.to_html(float_format=lambda x: f'{x:.4f}')

print("\nHTML with Custom Styling Generated!")
print("="*70)
print("Includes:")
print("- Professional font family")
print("- Box shadow for depth")
print("- Hover effects")
print("- Clean styling")

# To save:
# with open('fe_results_styled.html', 'w') as f:
#     f.write(html_styled)


HTML with Custom Styling Generated!
Includes:
- Professional font family
- Box shadow for depth
- Hover effects
- Clean styling


### 3.3 Displaying in Jupyter

In Jupyter notebooks, you can display HTML directly:

In [8]:
# Display styled HTML table in Jupyter
from IPython.display import HTML, display

# Add significance stars to p-values
def add_stars(row):
    pval = row['P-value']
    if pval < 0.001:
        stars = '***'
    elif pval < 0.01:
        stars = '**'
    elif pval < 0.05:
        stars = '*'
    else:
        stars = ''
    return stars

results_display = results_df.copy()
results_display['Sig'] = results_df.apply(add_stars, axis=1)

# Style the dataframe
styled_df = results_display.style.format({
    'Coefficient': '{:.4f}',
    'Std Error': '{:.4f}',
    't-statistic': '{:.3f}',
    'P-value': '{:.4f}'
}).set_caption("Fixed Effects Model Results")

print("\nStyled DataFrame for Jupyter:")
print("="*70)
display(styled_df)


Styled DataFrame for Jupyter:


,Coefficient,Std Error,t-statistic,P-value,Sig
value,0.1066,0.0113,9.401,0.0000,***
capital,0.3444,0.0178,19.397,0.0000,***


---

## 4. Markdown Export {#markdown}

**Markdown** is perfect for:
- GitHub READMEs
- Documentation  
- Version control (git-friendly)
- Simple sharing

### 4.1 Basic Markdown Table

Use pandas to create markdown tables:

In [9]:
# Generate Markdown using pandas
# Note: requires 'tabulate' package: pip install tabulate
try:
    markdown_output = results_df.to_markdown(floatfmt=".4f")
    print("MARKDOWN OUTPUT")
    print("="*70)
    print(markdown_output)
    print("\n💡 Copy-paste this directly into:")
    print("- README.md files")
    print("- GitHub issues/PRs")
    print("- Documentation")
    print("- Jupyter markdown cells")
except ImportError:
    print("MARKDOWN OUTPUT")
    print("="*70)
    print("Note: Install 'tabulate' package for markdown export:")
    print("  pip install tabulate")
    print("\nAlternatively, create markdown tables manually:")
    print(f"| Variable | Coefficient | Std Error | t-statistic | P-value |")
    print(f"|----------|-------------|-----------|-------------|---------|")
    for var in results_df.index:
        print(f"| {var} | {results_df.loc[var, 'Coefficient']:.4f} | "
              f"{results_df.loc[var, 'Std Error']:.4f} | "
              f"{results_df.loc[var, 't-statistic']:.3f} | "
              f"{results_df.loc[var, 'P-value']:.4f} |")

MARKDOWN OUTPUT
Note: Install 'tabulate' package for markdown export:
  pip install tabulate

Alternatively, create markdown tables manually:
| Variable | Coefficient | Std Error | t-statistic | P-value |
|----------|-------------|-----------|-------------|---------|
| value | 0.1066 | 0.0113 | 9.401 | 0.0000 |
| capital | 0.3444 | 0.0178 | 19.397 | 0.0000 |


### 4.2 Markdown Comparison Table

In [10]:
# Comparison table in Markdown
comparison_md = f"""
# Model Comparison: Grunfeld Investment Data

| Variable | Pooled OLS | Fixed Effects | Random Effects |
|----------|------------|---------------|----------------|
| value    | {pooled_results.params['value']:.4f} | {fe_results.params['value']:.4f} | {re_results.params['value']:.4f} |
|          | ({pooled_results.std_errors['value']:.4f}) | ({fe_results.std_errors['value']:.4f}) | ({re_results.std_errors['value']:.4f}) |
| capital  | {pooled_results.params['capital']:.4f} | {fe_results.params['capital']:.4f} | {re_results.params['capital']:.4f} |
|          | ({pooled_results.std_errors['capital']:.4f}) | ({fe_results.std_errors['capital']:.4f}) | ({re_results.std_errors['capital']:.4f}) |
| | | | |
| N        | {pooled_results.nobs} | {fe_results.nobs} | {re_results.nobs} |
| R²       | {pooled_results.rsquared:.4f} | {fe_results.rsquared_within:.4f} | {re_results.rsquared:.4f} |

*Standard errors in parentheses*
"""

print("MARKDOWN COMPARISON TABLE")
print("="*70)
print(comparison_md)

# Save to file:
# with open('model_comparison.md', 'w') as f:
#     f.write(comparison_md)

MARKDOWN COMPARISON TABLE

# Model Comparison: Grunfeld Investment Data

| Variable | Pooled OLS | Fixed Effects | Random Effects |
|----------|------------|---------------|----------------|
| value    | 0.0951 | 0.1066 | 0.1006 |
|          | (0.0072) | (0.0113) | (0.0114) |
| capital  | 0.2940 | 0.3444 | 0.3444 |
|          | (0.0327) | (0.0178) | (0.0176) |
| | | | |
| N        | 200 | 200 | 200 |
| R²       | 0.8090 | 0.7899 | 0.8003 |

*Standard errors in parentheses*



### 4.3 Markdown with Badges/Emojis

Make it engaging with badges and emojis:

In [11]:
# Enhanced Markdown
# Create markdown from results DataFrame
md_df = results_df.copy()

# Create markdown table (with fallback if tabulate not installed)
try:
    table_md = md_df.to_markdown(floatfmt=".4f")
except ImportError:
    # Manual markdown table if tabulate not available
    table_md = "| Variable | Coefficient | Std Error | t-statistic | P-value |\n"
    table_md += "|----------|-------------|-----------|-------------|---------|"
    for var in md_df.index:
        table_md += f"\n| {var} | {md_df.loc[var, 'Coefficient']:.4f} | "
        table_md += f"{md_df.loc[var, 'Std Error']:.4f} | "
        table_md += f"{md_df.loc[var, 't-statistic']:.3f} | "
        table_md += f"{md_df.loc[var, 'P-value']:.4f} |"

enhanced_md = f"""# 📊 Fixed Effects Model Results

![Status](https://img.shields.io/badge/Status-Complete-green)
![R²](https://img.shields.io/badge/R²-{fe_results.rsquared_within:.3f}-blue)
![N](https://img.shields.io/badge/N-{fe_results.nobs}-orange)

## Results

{table_md}

## Interpretation

- ✅ Both coefficients are **highly significant** (p < 0.001)
- 📈 Value coefficient: {fe_results.params['value']:.4f}
  - A 1-unit increase in firm value increases investment by {fe_results.params['value']:.2f}
- 🏭 Capital coefficient: {fe_results.params['capital']:.4f}
  - A 1-unit increase in capital stock increases investment by {fe_results.params['capital']:.2f}

## Model Fit

- R² (within): {fe_results.rsquared_within:.4f}
- Observations: {fe_results.nobs}
- Entities: {fe_results.n_entities}

---

*Generated with PanelBox v{pb.__version__}*
"""

print("ENHANCED MARKDOWN")
print("="*70)
print(enhanced_md)

ENHANCED MARKDOWN
# 📊 Fixed Effects Model Results

![Status](https://img.shields.io/badge/Status-Complete-green)
![R²](https://img.shields.io/badge/R²-0.790-blue)
![N](https://img.shields.io/badge/N-200-orange)

## Results

| Variable | Coefficient | Std Error | t-statistic | P-value |
|----------|-------------|-----------|-------------|---------|
| value | 0.1066 | 0.0113 | 9.401 | 0.0000 |
| capital | 0.3444 | 0.0178 | 19.397 | 0.0000 |

## Interpretation

- ✅ Both coefficients are **highly significant** (p < 0.001)
- 📈 Value coefficient: 0.1066
  - A 1-unit increase in firm value increases investment by 0.11
- 🏭 Capital coefficient: 0.3444
  - A 1-unit increase in capital stock increases investment by 0.34

## Model Fit

- R² (within): 0.7899
- Observations: 200
- Entities: 10

---

*Generated with PanelBox v0.8.0*



---

## 5. LaTeX Tables {#latex}

**LaTeX** for academic papers and journals.

### 5.1 Basic LaTeX Table

Use pandas to create LaTeX tables:

In [12]:
# Generate LaTeX using pandas
latex_output = results_df.to_latex(float_format="%.4f")

print("LaTeX OUTPUT")
print("="*70)
print(latex_output)
print("\n💡 Include in your paper:")
print("\\input{fe_results.tex}")
print("Or copy-paste the table into your .tex file")

# Save to file:
# with open('fe_results.tex', 'w') as f:
#     f.write(latex_output)

LaTeX OUTPUT
\begin{tabular}{lrrrr}
\toprule
 & Coefficient & Std Error & t-statistic & P-value \\
\midrule
value & 0.1066 & 0.0113 & 9.4014 & 0.0000 \\
capital & 0.3444 & 0.0178 & 19.3974 & 0.0000 \\
\bottomrule
\end{tabular}


💡 Include in your paper:
\input{fe_results.tex}
Or copy-paste the table into your .tex file


### 5.2 LaTeX with Custom Formatting

Add caption, label, and custom formatting:

In [13]:
# Custom LaTeX formatting
latex_custom = results_df.to_latex(
    caption="Fixed Effects Model: Grunfeld Investment Data",
    label="tab:fe_results",
    float_format="%.4f",
    column_format="lcccc"
)

print("\nCUSTOM LaTeX TABLE")
print("="*70)
print(latex_custom)
print("\n💡 Features:")
print("- Custom caption")
print("- Label for referencing (\\ref{tab:fe_results})")
print("- Custom number formatting")
print("- Column alignment")


CUSTOM LaTeX TABLE
\begin{table}
\caption{Fixed Effects Model: Grunfeld Investment Data}
\label{tab:fe_results}
\begin{tabular}{lcccc}
\toprule
 & Coefficient & Std Error & t-statistic & P-value \\
\midrule
value & 0.1066 & 0.0113 & 9.4014 & 0.0000 \\
capital & 0.3444 & 0.0178 & 19.3974 & 0.0000 \\
\bottomrule
\end{tabular}
\end{table}


💡 Features:
- Custom caption
- Label for referencing (\ref{tab:fe_results})
- Custom number formatting
- Column alignment


### 5.3 LaTeX Comparison Table (Multiple Models)

Create a comparison table manually:

In [14]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Pooled OLS': [
        f"{pooled_results.params['value']:.4f}",
        f"({pooled_results.std_errors['value']:.4f})",
        f"{pooled_results.params['capital']:.4f}",
        f"({pooled_results.std_errors['capital']:.4f})",
        pooled_results.nobs,
        f"{pooled_results.rsquared:.4f}"
    ],
    'Fixed Effects': [
        f"{fe_results.params['value']:.4f}",
        f"({fe_results.std_errors['value']:.4f})",
        f"{fe_results.params['capital']:.4f}",
        f"({fe_results.std_errors['capital']:.4f})",
        fe_results.nobs,
        f"{fe_results.rsquared_within:.4f}"
    ],
    'Random Effects': [
        f"{re_results.params['value']:.4f}",
        f"({re_results.std_errors['value']:.4f})",
        f"{re_results.params['capital']:.4f}",
        f"({re_results.std_errors['capital']:.4f})",
        re_results.nobs,
        f"{re_results.rsquared:.4f}"
    ]
}, index=['value', '', 'capital', '', 'N', 'R²'])

# Export to LaTeX
latex_comparison = comparison_df.to_latex(
    caption="Comparison of Panel Data Models",
    label="tab:model_comparison"
)

print("\nLaTeX COMPARISON TABLE")
print("="*70)
print(latex_comparison)
print("\n💡 This table is ready for:")
print("- Journal submissions")
print("- Working papers")
print("- Dissertations")
print("- Conference presentations")


LaTeX COMPARISON TABLE
\begin{table}
\caption{Comparison of Panel Data Models}
\label{tab:model_comparison}
\begin{tabular}{llll}
\toprule
 & Pooled OLS & Fixed Effects & Random Effects \\
\midrule
value & 0.0951 & 0.1066 & 0.1006 \\
 & (0.0072) & (0.0113) & (0.0114) \\
capital & 0.2940 & 0.3444 & 0.3444 \\
 & (0.0327) & (0.0178) & (0.0176) \\
N & 200 & 200 & 200 \\
R² & 0.8090 & 0.7899 & 0.8003 \\
\bottomrule
\end{tabular}
\end{table}


💡 This table is ready for:
- Journal submissions
- Working papers
- Dissertations
- Conference presentations


### 5.4 LaTeX Table for Econometrics Journals

Standard format used in top journals:

In [15]:
# Journal-style table
journal_latex = f"""
\\begin{{table}}[htbp]
\\centering
\\caption{{Investment Equation Estimates}}
\\label{{tab:investment}}
\\begin{{tabular}}{{l*{{4}}{{c}}}}
\\hline\\hline
                    & \\multicolumn{{4}}{{c}}{{Dependent variable: Investment}} \\\\
\\cmidrule{{2-5}}
                    & (1) & (2) & (3) & (4) \\\\
                    & Pooled & FE & RE & FE-Robust \\\\
\\hline
Value               & {pooled_results.params['value']:.3f}$^{{***}}$ & {fe_results.params['value']:.3f}$^{{***}}$ & {re_results.params['value']:.3f}$^{{***}}$ & {fe_robust_results.params['value']:.3f}$^{{***}}$ \\\\
                    & ({pooled_results.std_errors['value']:.3f}) & ({fe_results.std_errors['value']:.3f}) & ({re_results.std_errors['value']:.3f}) & ({fe_robust_results.std_errors['value']:.3f}) \\\\
[0.5em]
Capital             & {pooled_results.params['capital']:.3f}$^{{***}}$ & {fe_results.params['capital']:.3f}$^{{**}}$ & {re_results.params['capital']:.3f}$^{{***}}$ & {fe_robust_results.params['capital']:.3f}$^{{**}}$ \\\\
                    & ({pooled_results.std_errors['capital']:.3f}) & ({fe_results.std_errors['capital']:.3f}) & ({re_results.std_errors['capital']:.3f}) & ({fe_robust_results.std_errors['capital']:.3f}) \\\\
\\hline
Observations        & {pooled_results.nobs} & {fe_results.nobs} & {re_results.nobs} & {fe_robust_results.nobs} \\\\
R$^2$               & {pooled_results.rsquared:.3f} & {fe_results.rsquared_within:.3f} & {re_results.rsquared:.3f} & {fe_robust_results.rsquared_within:.3f} \\\\
Firm FE             & No & Yes & No & Yes \\\\
Clustered SE        & No & No & No & Yes \\\\
\\hline\\hline
\\end{{tabular}}
\\\\[1em]
\\begin{{minipage}}{{0.8\\textwidth}}
{{\\footnotesize \\textit{{Notes:}} Standard errors in parentheses. 
$^{{***}}$ p$<$0.01, $^{{**}}$ p$<$0.05, $^{{*}}$ p$<$0.1. 
Column (4) uses standard errors clustered by firm. 
Sample: 10 firms observed over 20 years (1935-1954).}}
\\end{{minipage}}
\\end{{table}}
"""

print("\nJOURNAL-STYLE LaTeX TABLE")
print("="*70)
print(journal_latex)

print("\n💡 This follows conventions from:")
print("- American Economic Review (AER)")
print("- Journal of Econometrics")
print("- Review of Economic Studies")
print("- And other top journals")


JOURNAL-STYLE LaTeX TABLE

\begin{table}[htbp]
\centering
\caption{Investment Equation Estimates}
\label{tab:investment}
\begin{tabular}{l*{4}{c}}
\hline\hline
                    & \multicolumn{4}{c}{Dependent variable: Investment} \\
\cmidrule{2-5}
                    & (1) & (2) & (3) & (4) \\
                    & Pooled & FE & RE & FE-Robust \\
\hline
Value               & 0.095$^{***}$ & 0.107$^{***}$ & 0.101$^{***}$ & 0.107$^{***}$ \\
                    & (0.007) & (0.011) & (0.011) & (0.014) \\
[0.5em]
Capital             & 0.294$^{***}$ & 0.344$^{**}$ & 0.344$^{***}$ & 0.344$^{**}$ \\
                    & (0.033) & (0.018) & (0.018) & (0.034) \\
\hline
Observations        & 200 & 200 & 200 & 200 \\
R$^2$               & 0.809 & 0.790 & 0.800 & 0.790 \\
Firm FE             & No & Yes & No & Yes \\
Clustered SE        & No & No & No & Yes \\
\hline\hline
\end{tabular}
\\[1em]
\begin{minipage}{0.8\textwidth}
{\footnotesize \textit{Notes:} Standard errors in parentheses. 
$^{**

---

## 6. Comparison Tables {#comparison}

Create side-by-side comparison of multiple models.

### 6.1 Simple Comparison DataFrame

In [16]:
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Pooled OLS': [
        f"{pooled_results.params['value']:.4f} ({pooled_results.std_errors['value']:.4f})",
        f"{pooled_results.params['capital']:.4f} ({pooled_results.std_errors['capital']:.4f})",
        pooled_results.nobs,
        f"{pooled_results.rsquared:.4f}"
    ],
    'Fixed Effects': [
        f"{fe_results.params['value']:.4f} ({fe_results.std_errors['value']:.4f})",
        f"{fe_results.params['capital']:.4f} ({fe_results.std_errors['capital']:.4f})",
        fe_results.nobs,
        f"{fe_results.rsquared_within:.4f}"
    ],
    'Random Effects': [
        f"{re_results.params['value']:.4f} ({re_results.std_errors['value']:.4f})",
        f"{re_results.params['capital']:.4f} ({re_results.std_errors['capital']:.4f})",
        re_results.nobs,
        f"{re_results.rsquared:.4f}"
    ]
}, index=['value', 'capital', 'N', 'R²'])

print("COMPARISON TABLE")
print("="*70)
print(comparison_df)

print("\nNote: Standard errors in parentheses")

COMPARISON TABLE
              Pooled OLS    Fixed Effects   Random Effects
value    0.0951 (0.0072)  0.1066 (0.0113)  0.1006 (0.0114)
capital  0.2940 (0.0327)  0.3444 (0.0178)  0.3444 (0.0176)
N                    200              200              200
R²                0.8090           0.7899           0.8003

Note: Standard errors in parentheses


### 6.2 Comparison with Test Statistics

In [17]:
# Enhanced comparison with p-values
def format_coef(params, stderr, pval):
    stars = ''
    if pval < 0.001:
        stars = '***'
    elif pval < 0.01:
        stars = '**'
    elif pval < 0.05:
        stars = '*'
    return f"{params:.4f}{stars}\n({stderr:.4f})"

comparison_enhanced = pd.DataFrame({
    'Pooled OLS': [
        format_coef(pooled_results.params['value'], 
                   pooled_results.std_errors['value'],
                   pooled_results.pvalues['value']),
        format_coef(pooled_results.params['capital'], 
                   pooled_results.std_errors['capital'],
                   pooled_results.pvalues['capital'])
    ],
    'Fixed Effects': [
        format_coef(fe_results.params['value'], 
                   fe_results.std_errors['value'],
                   fe_results.pvalues['value']),
        format_coef(fe_results.params['capital'], 
                   fe_results.std_errors['capital'],
                   fe_results.pvalues['capital'])
    ],
    'Random Effects': [
        format_coef(re_results.params['value'], 
                   re_results.std_errors['value'],
                   re_results.pvalues['value']),
        format_coef(re_results.params['capital'], 
                   re_results.std_errors['capital'],
                   re_results.pvalues['capital'])
    ]
}, index=['value', 'capital'])

print("\nCOMPARISON WITH SIGNIFICANCE STARS")
print("="*70)
print(comparison_enhanced)
print("\n*** p<0.001, ** p<0.01, * p<0.05")
print("Standard errors in parentheses")


COMPARISON WITH SIGNIFICANCE STARS
                  Pooled OLS        Fixed Effects       Random Effects
value    0.0951***\n(0.0072)  0.1066***\n(0.0113)  0.1006***\n(0.0114)
capital  0.2940***\n(0.0327)  0.3444***\n(0.0178)  0.3444***\n(0.0176)

*** p<0.001, ** p<0.01, * p<0.05
Standard errors in parentheses


---

## 7. Custom Formatting {#custom}

### 7.1 Format Numbers

Control decimal places and notation:

In [18]:
# Custom number formatting
formatted_results = pd.DataFrame({
    'Coefficient': fe_results.params.apply(lambda x: f"{x:.6f}"),
    'Std Error': fe_results.std_errors.apply(lambda x: f"{x:.6f}"),
    't-stat': fe_results.tvalues.apply(lambda x: f"{x:.3f}"),
    'P-value': fe_results.pvalues.apply(lambda x: f"{x:.4e}" if x < 0.001 else f"{x:.4f}")
})

print("CUSTOM FORMATTED RESULTS")
print("="*70)
print(formatted_results)

print("\n💡 Customizations:")
print("- 6 decimals for coefficients")
print("- 3 decimals for t-stats")
print("- Scientific notation for p < 0.001")

CUSTOM FORMATTED RESULTS
        Coefficient Std Error  t-stat     P-value
value      0.106585  0.011337   9.401  0.0000e+00
capital    0.344400  0.017755  19.397  0.0000e+00

💡 Customizations:
- 6 decimals for coefficients
- 3 decimals for t-stats
- Scientific notation for p < 0.001


### 7.2 Color-Coded Output

Add color coding for significance:

In [19]:
# Color-coded significance (for terminal/notebooks)
def color_pvalue(pval):
    if pval < 0.001:
        return f"\033[92m{pval:.4f}***\033[0m"  # Green
    elif pval < 0.01:
        return f"\033[93m{pval:.4f}**\033[0m"   # Yellow
    elif pval < 0.05:
        return f"\033[94m{pval:.4f}*\033[0m"    # Blue
    else:
        return f"{pval:.4f}"

print("\nCOLOR-CODED P-VALUES")
print("="*70)
for var in fe_results.params.index:
    pval = fe_results.pvalues[var]
    print(f"{var:15s}: {color_pvalue(pval)}")

print("\nGreen=***, Yellow=**, Blue=*, Black=n.s.")


COLOR-CODED P-VALUES
value          : 0.0000***
capital        : 0.0000***

Green=***, Yellow=**, Blue=*, Black=n.s.


---

## 8. Automated Workflows {#automation}

### 8.1 Batch Report Generation

Generate reports for multiple models automatically:

In [20]:
# Automated reporting function
def generate_all_reports(results_dict, output_dir='reports/'):
    """
    Generate HTML, Markdown, and LaTeX for all models.
    
    Parameters
    ----------
    results_dict : dict
        {model_name: results_object}
    output_dir : str
        Directory to save reports
    """
    import os
    
    # Create directory if doesn't exist
    # os.makedirs(output_dir, exist_ok=True)
    
    for name, results in results_dict.items():
        # Create DataFrame from results
        df = pd.DataFrame({
            'Coefficient': results.params,
            'Std Error': results.std_errors,
            't-statistic': results.tvalues,
            'P-value': results.pvalues
        })
        
        # Generate all formats
        html = df.to_html(float_format=lambda x: f'{x:.4f}')
        
        # Markdown with fallback
        try:
            md = df.to_markdown(floatfmt=".4f")
        except ImportError:
            md = "# Results\n\nInstall 'tabulate' for markdown export: pip install tabulate"
        
        latex = df.to_latex(float_format="%.4f")
        
        # Save files (commented out for demo)
        # with open(f'{output_dir}{name}_report.html', 'w') as f:
        #     f.write(html)
        # with open(f'{output_dir}{name}_report.md', 'w') as f:
        #     f.write(md)
        # with open(f'{output_dir}{name}_table.tex', 'w') as f:
        #     f.write(latex)
        
        print(f"✓ Generated reports for: {name}")
    
    print(f"\n✅ All reports would be saved to: {output_dir}")

# Usage
models = {
    'pooled': pooled_results,
    'fe': fe_results,
    're': re_results,
    'fe_robust': fe_robust_results
}

print("AUTOMATED REPORT GENERATION")
print("="*70)
generate_all_reports(models)
print("\n💡 In practice, uncomment the file writing lines to save!")

AUTOMATED REPORT GENERATION
✓ Generated reports for: pooled
✓ Generated reports for: fe
✓ Generated reports for: re
✓ Generated reports for: fe_robust

✅ All reports would be saved to: reports/

💡 In practice, uncomment the file writing lines to save!


### 8.2 Report Template Function

Create a reusable template:

In [21]:
# Reusable report template
def create_panel_report(results, model_name, output_format='markdown'):
    """
    Create standardized report with custom header/footer.
    
    Parameters
    ----------
    results : PanelResults
        Estimation results
    model_name : str
        Name of the model
    output_format : str
        'html', 'markdown', or 'latex'
    
    Returns
    -------
    str
        Formatted report
    """
    header = f"""
========================================
Panel Data Analysis Report
========================================
Model: {model_name}
Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}
PanelBox Version: {pb.__version__}
========================================
"""
    
    # Create DataFrame from results
    df = pd.DataFrame({
        'Coefficient': results.params,
        'Std Error': results.std_errors,
        't-statistic': results.tvalues,
        'P-value': results.pvalues
    })
    
    if output_format == 'html':
        content = df.to_html(float_format=lambda x: f'{x:.4f}')
    elif output_format == 'markdown':
        try:
            content = df.to_markdown(floatfmt=".4f")
        except ImportError:
            content = "Install 'tabulate' for markdown export: pip install tabulate"
    elif output_format == 'latex':
        content = df.to_latex(float_format="%.4f")
    else:
        raise ValueError("Format must be 'html', 'markdown', or 'latex'")
    
    footer = """
========================================
Generated automatically with PanelBox
https://github.com/PanelBox-Econometrics-Model/panelbox
========================================
"""
    
    return header + "\n" + content + "\n" + footer

# Usage
report = create_panel_report(fe_results, "Fixed Effects Model", "markdown")

print("STANDARDIZED REPORT")
print("="*70)
print(report)

STANDARDIZED REPORT

Panel Data Analysis Report
Model: Fixed Effects Model
Date: 2026-02-12 18:55
PanelBox Version: 0.8.0

Install 'tabulate' for markdown export: pip install tabulate

Generated automatically with PanelBox
https://github.com/PanelBox-Econometrics-Model/panelbox



### 8.3 Email-Ready Report

Format report for email distribution:

In [22]:
# Email-friendly report
def create_email_report(results_list, model_names, subject="Panel Analysis Results"):
    """Create plain text report suitable for email."""
    
    email_body = f"""
Subject: {subject}

Hello,

Please find the panel data analysis results below.

{'='*60}
SUMMARY OF MODELS
{'='*60}
"""
    
    for i, (results, name) in enumerate(zip(results_list, model_names), 1):
        email_body += f"""
\nMODEL {i}: {name}
{'-'*60}
N = {results.nobs}
R² = {results.rsquared:.4f}

Key Coefficients:
"""
        for var in results.params.index[:5]:  # Show first 5
            sig = ""
            if results.pvalues[var] < 0.001:
                sig = " ***"
            elif results.pvalues[var] < 0.01:
                sig = " **"
            elif results.pvalues[var] < 0.05:
                sig = " *"
                
            email_body += f"  {var}: {results.params[var]:.4f} ({results.std_errors[var]:.4f}){sig}\n"
    
    email_body += f"""
\n{'='*60}
*** p<0.001, ** p<0.01, * p<0.05
Standard errors in parentheses

Full detailed reports attached separately.

Best regards,
Your Analysis Team
"""
    
    return email_body

# Create email report
email_report = create_email_report(
    [pooled_results, fe_results, re_results],
    ['Pooled OLS', 'Fixed Effects', 'Random Effects']
)

print("EMAIL-READY REPORT")
print("="*70)
print(email_report)

print("\n💡 Copy-paste this into your email!")

EMAIL-READY REPORT

Subject: Panel Analysis Results

Hello,

Please find the panel data analysis results below.

SUMMARY OF MODELS


MODEL 1: Pooled OLS
------------------------------------------------------------
N = 200
R² = 0.8090

Key Coefficients:
  Intercept: -10.2135 (8.7006)
  value: 0.0951 (0.0072) ***
  capital: 0.2940 (0.0327) ***


MODEL 2: Fixed Effects
------------------------------------------------------------
N = 200
R² = 0.7899

Key Coefficients:
  value: 0.1066 (0.0113) ***
  capital: 0.3444 (0.0178) ***


MODEL 3: Random Effects
------------------------------------------------------------
N = 200
R² = 0.8003

Key Coefficients:
  Intercept: -25.3638 (0.0294) ***
  value: 0.1006 (0.0114) ***
  capital: 0.3444 (0.0176) ***


*** p<0.001, ** p<0.01, * p<0.05
Standard errors in parentheses

Full detailed reports attached separately.

Best regards,
Your Analysis Team


💡 Copy-paste this into your email!


---

## Summary

You learned:

✅ **Basic Reporting**: `.summary()` method and DataFrame extraction  
✅ **HTML Export**: Using pandas to create HTML tables with custom styling  
✅ **Markdown Export**: GitHub-friendly tables with badges and formatting  
✅ **LaTeX Tables**: Publication-ready tables for academic papers  
✅ **Comparison Tables**: Side-by-side model comparison using DataFrames  
✅ **Custom Formatting**: Numbers, colors, significance stars  
✅ **Automation**: Batch generation, templates, email reports

### Key Takeaways

1. **Choose format by audience**:
   - HTML for exploration/web
   - Markdown for GitHub/docs
   - LaTeX for papers

2. **Always include**:
   - Standard errors
   - Significance stars
   - Model statistics (N, R²)
   - Notes/caption

3. **Automate** repetitive reporting tasks

4. **Use pandas** for flexible export options

### Best Practices

✅ **Document everything** in captions/notes  
✅ **Use significance stars** (standard convention)  
✅ **Show standard errors** (usually in parentheses)  
✅ **Compare models** side-by-side  
✅ **Automate** for reproducibility  
✅ **Version control** your reports (especially Markdown/LaTeX)

### Next Steps

Now you can:
- Create professional reports using pandas export functions
- Export to any format needed (HTML, Markdown, LaTeX)
- Automate report generation workflows
- Customize formatting to your preferences

**Explore other notebooks**:
- **[03_validation_complete.ipynb](./03_validation_complete.ipynb)**: Comprehensive testing
- **[04_robust_inference.ipynb](./04_robust_inference.ipynb)**: Robust methods
- **[02_dynamic_gmm_complete.ipynb](./02_dynamic_gmm_complete.ipynb)**: Dynamic panels

---

*Create beautiful reports with PanelBox!*